In [5]:
import spotipy
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import re
import math
from tqdm import tqdm

load_dotenv('../secrets/.env', override=True)

scope = "playlist-modify-private"

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

my_id = spotify.current_user()['id']

%run useful_functions.ipynb
%run spotify_scraping.ipynb
%run spotify_scraping_playlists.ipynb

In [4]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()
search = get_spotify_search()
playlists = get_spotify_playlists()
playlist_tracks = get_spotify_playlist_tracks()

master_df = (
broadcast_playlists
.merge(broadcast_details, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
.merge(shows, on='Show Key', how='left')
.merge(search,left_on=['Artist Name','Track Name'], right_on=['Original Search Artist','Original Search Track'], how = "left", suffixes=("", " Search"))
.merge(tracks, on='Track ID', how='left', suffixes=("", " Tracks"))
.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
)

del(shows,
    show_broadcast_details,
    broadcast_details,
    broadcast_playlists,
    search)

#set(master_df['Show Name'])

In [6]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Indie Show with Jack Saunders")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Indie Show with Jack Saunders"
description = "Songs from the Indie Show with Jack Saunders in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,57jJsS1qNk12kOUHcRpJMm,2KZedcpHb83f2AXlBDprAV,0
0,1UOkwtqARzu9QW4IMQjeO0,2KZedcpHb83f2AXlBDprAV,1
0,72zUxRL6AKWG67No7GmZuc,2KZedcpHb83f2AXlBDprAV,2
0,4m6CLhBh46R33ELYBRa7ge,2KZedcpHb83f2AXlBDprAV,3
0,02oTRx0v3GloOe5VJZgjC7,2KZedcpHb83f2AXlBDprAV,4
...,...,...,...
0,2mdaMDbjFxmxQPO90PxT5w,2KZedcpHb83f2AXlBDprAV,729
0,2JjDl1YbzaBKlhSrUXZqNM,2KZedcpHb83f2AXlBDprAV,730
0,0AIlxnJ1kFpu6wp89rjjUv,2KZedcpHb83f2AXlBDprAV,731
0,7BZWTu4Pv77TxGmmsQaSXN,2KZedcpHb83f2AXlBDprAV,732


In [7]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Sounds with Clara Amfo")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Sounds with Clara Amfo"
description = "Songs from Future Sounds with Clara Amfo in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,4wwZS4a7MR2DZIRjq8G5lD,3VTYIgusziIIdXiWB5hB2f,0
0,1fDFHXcykq4iw8Gg7s5hG9,3VTYIgusziIIdXiWB5hB2f,1
0,0hJfuyUwtEYMlGgvr3nzz9,3VTYIgusziIIdXiWB5hB2f,2
0,1qYSKTn4Peb38J5828xUmF,3VTYIgusziIIdXiWB5hB2f,3
0,4lxezjHzTpF81nMNixy1OS,3VTYIgusziIIdXiWB5hB2f,4
...,...,...,...
0,74RWuKK8uKMbxj4KnJpEgG,3VTYIgusziIIdXiWB5hB2f,463
0,6UoIrO15w3w62Jj8kGCnR7,3VTYIgusziIIdXiWB5hB2f,464
0,5WnvDdcyrFjbIqRxiqlet1,3VTYIgusziIIdXiWB5hB2f,465
0,6JkuJREwd6rd7t1YN069LK,3VTYIgusziIIdXiWB5hB2f,466


In [8]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Artists with Jack Saunders")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Artists with Jack Saunders"
description = "Songs from Future Artists with Jack Saunders in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,7caD4SRq2jQfbFy2ZfT3iz,3Wo1IZcPFAfrQJzBpW8QbG,0
0,2idpJZnXSvfURT7xk9iYF0,3Wo1IZcPFAfrQJzBpW8QbG,1
0,29Ml8I5tBgbYJW2cdaLJ1A,3Wo1IZcPFAfrQJzBpW8QbG,2
0,3GJbXOYweOltxSiOsCNkMd,3Wo1IZcPFAfrQJzBpW8QbG,3
0,6mfM11rZkoXsU5xAArF4Ix,3Wo1IZcPFAfrQJzBpW8QbG,4
...,...,...,...
0,4IqwX48XrtkoqvZw4hxEfa,3Wo1IZcPFAfrQJzBpW8QbG,774
0,2vT3lchY4bU1BwM69CtDyj,3Wo1IZcPFAfrQJzBpW8QbG,775
0,5BUvQtKdYrWJtdYaSoyFfG,3Wo1IZcPFAfrQJzBpW8QbG,776
0,5cRhrmpBwkZjJ7eFMJKbVM,3Wo1IZcPFAfrQJzBpW8QbG,777


In [9]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Alternative")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Alternative"
description = "Songs from Future Alternative Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,60UCABiHLHRDrBqCT0YZmF,0U2xXmHS8QvPrX2WqGT1UH,0
0,5SDhI2jKg0S5fzEEqPCHiV,0U2xXmHS8QvPrX2WqGT1UH,1
0,1aLuV1aBQBPzXKweNN1yyb,0U2xXmHS8QvPrX2WqGT1UH,2
0,48m7Kit1AFXmJuH244X4x7,0U2xXmHS8QvPrX2WqGT1UH,3
0,0bDFmPqNCXQcfAc3njZszA,0U2xXmHS8QvPrX2WqGT1UH,4
...,...,...,...
0,5QV6DCj4kvteYBDD9Vt1jY,0U2xXmHS8QvPrX2WqGT1UH,508
0,6L4L6bQPHojlmwBZQLQZjc,0U2xXmHS8QvPrX2WqGT1UH,509
0,5P0iRet3L91fC6SipKKsPl,0U2xXmHS8QvPrX2WqGT1UH,510
0,4ZG0EaIiFpFOaJKmT1ZjNO,0U2xXmHS8QvPrX2WqGT1UH,511


In [10]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Steve Lamacq")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Steve Lamacq"
description = "Songs from Steve Lamacq's Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,7sPEGLqEhyHDLGrz4EeLm2,6Bu0jMJgdN0Emarjx98IL7,0
0,3dfMqkZsvmoBFgyEe8878r,6Bu0jMJgdN0Emarjx98IL7,1
0,6x67AI64nGZtGflkHtZKay,6Bu0jMJgdN0Emarjx98IL7,2
0,4QGJabg9EfIsVdPfDO522b,6Bu0jMJgdN0Emarjx98IL7,3
0,3vZO25GdYuqFrR1kzZADnp,6Bu0jMJgdN0Emarjx98IL7,4
...,...,...,...
0,0mF5QmsOJq5w3FqGMjycII,6Bu0jMJgdN0Emarjx98IL7,1297
0,1ZtE8ysoPZVrMrQ24fVO4f,6Bu0jMJgdN0Emarjx98IL7,1298
0,2AmkpCt3y9NBsB5UgP4S6I,6Bu0jMJgdN0Emarjx98IL7,1299
0,5tmkX2fJ7KUAnMz3W1ibEr,6Bu0jMJgdN0Emarjx98IL7,1300


In [11]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == 'New Music Fix')].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = 'New Music Fix'
description = "Songs from New Music Fix in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,51aJzDWN1arzLTJjjwNXI5,0Cwr3XcLdGZ5ZqDtI7lL11,0
0,0R3wEiSEO8AM3UJBvdvvmv,0Cwr3XcLdGZ5ZqDtI7lL11,1
0,1ZtE8ysoPZVrMrQ24fVO4f,0Cwr3XcLdGZ5ZqDtI7lL11,2
0,3lQPBzcMsqmNldx9AgIhSt,0Cwr3XcLdGZ5ZqDtI7lL11,3
0,02zPsXmSUP6Rr1Z2w5KMpf,0Cwr3XcLdGZ5ZqDtI7lL11,4
...,...,...,...
0,1pxLVdC8sjLDA2DF3uOEuW,0Cwr3XcLdGZ5ZqDtI7lL11,613
0,0RTuA2AWVjFw0MURes4kse,0Cwr3XcLdGZ5ZqDtI7lL11,614
0,1MCrtBVXxy2p48uWwnBwEz,0Cwr3XcLdGZ5ZqDtI7lL11,615
0,7xfwMBuatcfbzTSWRfo49f,0Cwr3XcLdGZ5ZqDtI7lL11,616


In [12]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Chillest Show")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Chillest Show"
description = "Songs from the Chillest Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,3GYlZ7tbxLOxe6ewMNVTkw,1p5mKMwcQVCeQNVkzpUZqJ,0
0,4bBXnC1wcOCAuo3zg57JBl,1p5mKMwcQVCeQNVkzpUZqJ,1
0,5NtmfJwkA0AXsp2WWHviPn,1p5mKMwcQVCeQNVkzpUZqJ,2
0,7lBNPNIewZf6d1DJjRtlj8,1p5mKMwcQVCeQNVkzpUZqJ,3
0,0icYUkm7H0I21NG2oTdpG8,1p5mKMwcQVCeQNVkzpUZqJ,4
...,...,...,...
0,6u5lHozM0oGj2l16lTF3KP,1p5mKMwcQVCeQNVkzpUZqJ,1081
0,7oAuqs6akGnPU3Tb00ZmyM,1p5mKMwcQVCeQNVkzpUZqJ,1082
0,38vxstSXciei0IuEO2MoGI,1p5mKMwcQVCeQNVkzpUZqJ,1083
0,2xOdFPQn17smnZrFCtFNSv,1p5mKMwcQVCeQNVkzpUZqJ,1084


In [13]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Power Down Playlist with Sian Eleri")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Power Down Playlist with Sian Eleri"
description = "Songs from the Power Down Playlist with Sian Eleri in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,1OLzbUhgmGf9KCXj2ncYVO,3Tqe2k7TlEB5OatSMKeppu,0
0,5LT0mW0LPOF4dn0EGlMW3a,3Tqe2k7TlEB5OatSMKeppu,1
0,4h4OkBTTvffNBeLJA2V1jr,3Tqe2k7TlEB5OatSMKeppu,2
0,3sP6EGqcYVmDy9UBStCnRR,3Tqe2k7TlEB5OatSMKeppu,3
0,4Us55B2hPn5eR66q9WoYO1,3Tqe2k7TlEB5OatSMKeppu,4
...,...,...,...
0,1TLHZcMQRMgW3N7hjh9Zlu,3Tqe2k7TlEB5OatSMKeppu,704
0,0tNuJpBgtE65diL6Q8Q7fI,3Tqe2k7TlEB5OatSMKeppu,705
0,3Z3wyUUPcKyw4nJkW2PqBA,3Tqe2k7TlEB5OatSMKeppu,706
0,3INuZoAMZsz0BOiEFTLmAu,3Tqe2k7TlEB5OatSMKeppu,707


In [14]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == 'BBC Introducing on Radio 1')].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = 'BBC Introducing on Radio 1'
description = "Songs from BBC Introducing on Radio 1 in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,5BYoZQEbn9MO3RrMUSV4DD,07B5mpuLVdVpgSycsqn4Qc,0
0,50aFetaKwC3pKTLyUWh7UZ,07B5mpuLVdVpgSycsqn4Qc,1
0,4bd7Xa4thhz7ZqY8F9BpwD,07B5mpuLVdVpgSycsqn4Qc,2
0,5JbT1O4xgaupqbsd2mwZKY,07B5mpuLVdVpgSycsqn4Qc,3
0,5htQGN8PpkpoeyPDaBoF5S,07B5mpuLVdVpgSycsqn4Qc,4
...,...,...,...
0,2kC9Ccwk1ILSzCN3IBROOx,07B5mpuLVdVpgSycsqn4Qc,660
0,4f5SAiUVc4WFvuxva3JhnR,07B5mpuLVdVpgSycsqn4Qc,661
0,3BOiQtFYI4EE27CyNwm9rU,07B5mpuLVdVpgSycsqn4Qc,662
0,62UTBl5LVd6HOCFvT57Axt,07B5mpuLVdVpgSycsqn4Qc,663


In [26]:
#make_playlist(title, description)
#edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

#edit_playlist_details(user=my_id, playlist_id = playlist_id, name = name', description = 'description')
#get_playlist(playlist_id, with_tracks = False)
#get_playlist(playlist_id = playlist_id)
#delete_playlist(playlist_id = playlist_id)

In [15]:
playlist_tracks.to_pickle('../data/spotify_playlist_tracks.pkl')
playlists.to_pickle('../data/spotify_playlists.pkl')
tracks.to_pickle('../data/spotify_tracks.pkl')
albums.to_pickle('../data/spotify_albums.pkl')
artists.to_pickle('../data/spotify_artists.pkl')